In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
from functools import partial
from multiprocessing import Pool

from fastdtw import fastdtw, dtw
from sklearn import svm
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

In [2]:
reduced = pd.read_csv("reduced.csv", index_col="date")
d = reduced.apply(np.log).diff()[1:]
data = []
for podatak in d["KO"]:
    data.append(podatak)
d.shape

(969, 368)

In [3]:
L = len(data)
l = 20
s = 1

X = np.stack([data[i:i + l] for i in range(L // 2 - l - s)])
Y = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(L // 2 - l - s)]).astype(np.float32)
##ako je sljedeći element veći od zadnjeg clana niza onda dobiva 1 inace 0

X_test = np.stack([data[i:i + l] for i in range(L // 2, L - l - s)])
Y_test = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(L // 2, L - l - s)]).astype(np.float32)
print(len(X[0]))
print(X)
print(Y_test)

20
[[ 0.01483671 -0.01876502  0.00343812 ... -0.00173719 -0.00373274
   0.00868815]
 [-0.01876502  0.00343812  0.01508544 ... -0.00373274  0.00868815
  -0.0022269 ]
 [ 0.00343812  0.01508544 -0.0055697  ...  0.00868815 -0.0022269
   0.0029681 ]
 ...
 [ 0.00320079  0.          0.01197326 ... -0.00146843 -0.00147059
  -0.00689149]
 [ 0.          0.01197326 -0.0038939  ... -0.00147059 -0.00689149
   0.00172733]
 [ 0.01197326 -0.0038939  -0.00293041 ... -0.00689149  0.00172733
   0.00859217]]
[0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0.
 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0.
 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1.
 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0.
 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1.
 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0

In [4]:
len(X), len(X_test), Y.mean(), Y_test.mean()

(463, 464, 0.49676025, 0.4827586)

In [5]:
n = len(X)
m = len(X_test)
kernel_train = np.empty((n, n))
kernel_test = np.empty((m, n))

In [6]:
t0 = time.time()
for i in range(n):
    for j in range(i+1):
        if i == j:
            kernel_train[i][j] = 0.0
        else:
            kernel_train[i][j] = kernel_train[j][i] = fastdtw(X[i], X[j], radius = 5, dist = 2)[0]


for i in range(m):
    for j in range(n):           
        kernel_test[i][j] = fastdtw(X_test[i], X[j], radius = 5, dist = 2)[0]
                      
print(time.time()-t0)

4914.994704246521


In [7]:
#np.save("kernel_train_univariant_KO.npy", kernel_train)

In [8]:
kernel_train = np.load("kernel_train_univariant_KO.npy")

In [9]:
#np.save("kernel_test_univariant_KO.npy", kernel_test)

In [10]:
kernel_test = np.load("kernel_test_univariant_KO.npy")

In [13]:
c = 10
gamma = 25
kernel = np.exp(-kernel_train/gamma)
testkernel = np.exp(-kernel_test/gamma)
clf = svm.SVC(kernel='precomputed', C=c)
clf.fit(kernel, Y)
res = clf.predict(kernel)
#train set
print('accuracy train: ', np.mean(res == Y))
print('Area under curve(AUC) train:', roc_auc_score(Y, res))
#test set
res = clf.predict(testkernel)
print('accuracy test: ', np.mean(res == Y_test))
print('Area under curve(AUC) test:', roc_auc_score(Y_test, res))

accuracy train:  0.7645788336933045
Area under curve(AUC) train: 0.7639671580518754
accuracy test:  0.7629310344827587
Area under curve(AUC) test: 0.7604166666666666
